In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/home/marco/phd/devel/exotic/exotic/')

import os
import math

import numpy as np
import pandas as pd
import xarray as xr
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from IPython.display import display, HTML

from lab import analysis
from lab.simulation import forcings

REF_RESP_PATH = [
    '/home/marco/phd/data/response/lorenz96/rk4/SF_8.0_1.0_0/',
    '/home/marco/phd/data/response/lorenz96/rk4/SF_8.0_-1.0_0/'            
]

SAVE_FIG = False

## Plot cumulative distribution approximation

In [ ]:
observable = 'energy'
QUANTILES_PATH = f'/home/marco/phd/data/obs/lorenz96/rk4/CF_8.0/quantiles/obs_lorenz96_rk4_CF_8.0_quantiles_{observable}.nc'

In [ ]:
quantiles_da = xr.open_dataarray(QUANTILES_PATH)
quantiles_df = pd.DataFrame(data={'order': quantiles_da.quantile_order, 'value': quantiles_da.values})
display(plt.plot(quantiles_df.value, quantiles_df.order))

In [ ]:
force_linear_coefficient = 0.05
deactivation_time = 100
forcing = f'LF_8.0_{force_linear_coefficient}_0_{deactivation_time}'

In [ ]:
obs_all = [f"{observable}_below_{format(np.round(quant, 2), '.2f')}q" for quant in np.arange(0, 1.001, 0.01)]
chi = {}
for obs in obs_all:
    response_p1 = xr.open_dataarray(os.path.join(
        REF_RESP_PATH[0],
        observable,
        f'response_lorenz96_rk4_{obs}_SF_8.0_1.0_0.nc'
    ))
    chi[obs] = analysis.compute_susceptibility(response_p1.values.squeeze())
resp_pred = {}
for obs in obs_all:
    resp_pred[obs] = analysis.compute_response(chi[obs], forcings.LinearForcing(
        linear_coefficient=force_linear_coefficient,
        deactivation_time = deactivation_time
    ))

#### Plot CDF

In [ ]:
quant_order_rest = np.array(quantiles_df.order)

resp = [resp_pred[obs][0:10000] for obs in obs_all]
resp = np.array(resp)

for col in range(0, resp.shape[1]):
    resp[:, col] = resp[:, col] + quant_order_rest

z = resp
y = quantiles_df.value
x = np.arange(0, 100, 0.01)
fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.update_layout(title='PDF evolution',
                  scene = dict(
                    xaxis_title='time',
                    yaxis_title='quantile value',
                    zaxis_title='quantile order'
                  ),
                  autosize=False, width=750, height=750)